# Trabalho Final de Introdução a Linguagens Estatísicas

**Aluno**: Heitor Vinicius de Paula Salatiel

**Matrícula**: 1185913


## 02 - Tratamento da base *linguas.csv*

Essa base descreve as diversas línguas dominadas por um determindo funcionário, identificado pela chave **perfil_id**, descrevendo o nome da língua (coluna **name**), e o nível de proficiência em cada uma delas (coluna **proficiency**). Sendo assim, há mais de um registro para o mesmo perfil no caso do funcionário falar várias línguas.

Apenas 90 dos 111 funcinários constam nessa base.

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

linguas = pd.read_csv("bases/linguas.csv")

# Análise inicial
print("Nº de registros:", len(linguas))
print("Nº de valores únicos de perfil_id:", linguas.perfil_id.nunique())
print("Nº de valores únicos de name:", linguas.name.nunique())
print("Nº de valores únicos de proficiency:", linguas.proficiency.nunique())
print("Valores faltantes em perfil_id:", linguas.perfil_id.isnull().sum())
print("Valores faltantes em name:", linguas.name.isnull().sum())
print("Valores faltantes em proficiency:", linguas.proficiency.isnull().sum())

print("\nValores de name:", linguas.name.unique())

print("\nValores de proficiency:",linguas.proficiency.unique())

Nº de registros: 211
Nº de valores únicos de perfil_id: 90
Nº de valores únicos de name: 21
Nº de valores únicos de proficiency: 5
Valores faltantes em perfil_id: 0
Valores faltantes em name: 0
Valores faltantes em proficiency: 23

Valores de name: ['Idioma\nEspanhol' 'Idioma\nInglês' 'Idioma\nPortuguês'
 'Idioma\nItaliano' 'Idioma\nEnglish' 'Idioma\nPortuguese'
 'Idioma\nSpanish' 'Idioma\nFrancês' 'Idioma\nFrench' 'Idioma\nÁrabe'
 'Idioma\nHolandês' 'Idioma\nInglês - Certificação TOEIC' 'Idioma\nAlemão'
 'Idioma\nJaponês' 'Idioma\nLIBRAS'
 'Idioma\nInglês - Intermediário - em curso' 'Idioma\nItalian'
 'Idioma\nAnglais' 'Idioma\nFrançais' 'Idioma\nPortugais'
 'Idioma\nInglês médio (1100-1500)']

Valores de proficiency: ['Nível avançado' 'Fluente ou nativo' nan 'Nível básico a intermediário'
 'Nível básico' 'Nível intermediário']


Na base original há 21 categorias para a coluna **name**. Em uma análise visual, percebe-se que os nomes de uma mesma língua apresentavam variações. Além disso, os nomes estão com um prefixo "Idioma\n" que pode ser retirado. Esse nomes foram tratados retirando o prefixo e fazendo substituição da variação pelo respectivo nome da língua em português. O resultado do tratamento é armazenado na nova coluna **nome**.

In [2]:
# Copia a coluna name para preservar o dado original
linguas['nome'] = linguas.name

# Deduplica os nomes de linguas com variações para ficarem todos em Português
linguas.nome = linguas.name.str.replace("Idioma\n", "") \
    .str.replace("Anglais", "Inglês") \
    .str.replace("English", "Inglês") \
    .str.replace("Français", "Francês") \
    .str.replace(".*Inglês.*", "Inglês") \
    .str.replace("Italian.*", "Italiano") \
    .str.replace("Portug.*", "Português") \
    .str.replace("Spanish", "Espanhol")

# Análise das colunas categórias name e proficiency após tratamento
print("Nº de valores únicos de name após tratamento:", linguas.nome.nunique())

Nº de valores únicos de name após tratamento: 11


A coluna **proficiency** apresenta 5 valores distintos, mas foi remapeada apenas para os valores Básico, Intermediário e Avançado, que é suficiente para a análise dos perfis. O resultado do seu tratamento é armazenado na nova coluna **nivel**.

In [3]:
# Copia a coluna proficiency para preserva o dado original
linguas['nivel'] = linguas.proficiency

# Troca os valores não informados de proficiência por "Não informado"
linguas.nivel.fillna("Não informado", inplace=True)

# Define que o nível de profiência da lingua "Inglês - Intermediário - em curso" é "Nível intermediário"
linguas.loc[linguas.name == "Inglês - Intermediário - em curso", "nivel"] = "Nível intermediário"

# Faz com que as proficiências sejam apenas básico | intermediário | avançado | não informado
linguas.loc[linguas.nivel.str.contains("intermediário", case=False), "nivel"] = "Intermediário"
linguas.loc[linguas.nivel.str.contains("básico", case=False), "nivel"] = "Básico"
linguas.loc[linguas.nivel.str.contains("fluente", case=False), "nivel"] = "Avançado"
linguas.loc[linguas.nivel.str.contains("avançado", case=False), "nivel"] = "Avançado"

print("Nº de valores únicos de nivel após tratamento:", linguas.nivel.nunique())
print("Valores faltantes em nivel:", linguas.nivel.isnull().sum())

Nº de valores únicos de nivel após tratamento: 4
Valores faltantes em nivel: 0


Algumas informações consolidadas por perfil podem ser extraídas dessa base. Será observado quantas linguas a pessoa fala (**qtd_linguas**), e também será criada uma informação booleana se a pessoa fala ou não português (**fala_portugues**), se a pessoa fala ou não inglês (**fala_ingles**), e se a pessoa fala ou não espanhol (**fala_espanhol**).

In [6]:
# Consolida quantas linguas a pessoa fala
qtd_linguas = linguas.groupby("perfil_id").nome.count().to_frame()
qtd_linguas = qtd_linguas.rename(index=int, columns={"nome":"qtd_linguas"})

# Descobre se a pessoa fala ou não português
linguas['fala_portugues'] = False
linguas.loc[linguas.nome=="Português",'fala_portugues'] = True
fala_portugues = linguas[['perfil_id','fala_portugues']]
fala_portugues = fala_portugues.sort_values(by="fala_portugues", ascending=False).drop_duplicates(subset="perfil_id", keep="first")
fala_portugues = fala_portugues.sort_values(by="perfil_id").set_index('perfil_id')

# Descobre se a pessoa fala ou não inglês
linguas['fala_ingles'] = False
linguas.loc[linguas.nome=="Inglês",'fala_ingles'] = True
fala_ingles = linguas[['perfil_id','fala_ingles']]
fala_ingles = fala_ingles.sort_values(by="fala_ingles", ascending=False).drop_duplicates(subset="perfil_id", keep="first")
fala_ingles = fala_ingles.sort_values(by="perfil_id").set_index('perfil_id')

# Descobre se a pessoa fala ou não espanhol
linguas['fala_espanhol'] = False
linguas.loc[linguas.nome=="Espanhol",'fala_espanhol'] = True
fala_espanhol = linguas[['perfil_id','fala_espanhol']]
fala_espanhol = fala_espanhol.sort_values(by="fala_espanhol", ascending=False).drop_duplicates(subset="perfil_id", keep="first")
fala_espanhol = fala_espanhol.sort_values(by="perfil_id").set_index('perfil_id')

linguas.head()

,name,proficiency,perfil_id,nome,nivel,fala_portugues,fala_ingles,fala_espanhol
0,Idioma\nEspanhol,Nível avançado,0,Espanhol,Avançado,False,False,True
1,Idioma\nInglês,Fluente ou nativo,0,Inglês,Avançado,False,True,False
2,Idioma\nPortuguês,Fluente ou nativo,0,Português,Avançado,True,False,False
3,Idioma\nInglês,NaN,1,Inglês,Não informado,False,True,False
4,Idioma\nItaliano,NaN,1,Italiano,Não informado,False,False,False


O resulto final do tratamento da base original será gravado na base **linguas-tratado.csv**.

Uma outra base consolidada com um registro por funcionário será gravada, **perfil-linguas.csv**, contendo as colunas: **perfil_id**, **fala_portugues**, **fala_ingles**, **fala_espanhol**.

In [7]:
# Grava um csv tratado
linguas_tratado = linguas[['perfil_id', 'nome', 'nivel']].copy().set_index('perfil_id')
linguas_tratado.to_csv('bases/linguas-tratado.csv')

# Grava o csv com as informações consolidadas de língua
linguas_perfil = pd.concat([qtd_linguas, fala_portugues, fala_ingles, fala_espanhol],axis=1)
linguas_perfil.to_csv('bases/perfil-linguas.csv')

### Navegação:
* [Voltar para a análise](00-analise-dos-dados.ipynb)
* [Continuar para o próximo documento](03-tratamento-base-formacao.ipynb)